In [49]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from DeepPurpose.pybiomed_helper import _GetPseudoAAC, CalculateAADipeptideComposition, \
calcPubChemFingerAll, CalculateConjointTriad, GetQuasiSequenceOrder
import torch
from torch.utils import data
from torch.autograd import Variable
try:
	from descriptastorus.descriptors import rdDescriptors, rdNormalizedDescriptors
except:
	raise ImportError("Please install pip install git+https://github.com/bp-kelley/descriptastorus.")
from DeepPurpose.chemutils import get_mol, atom_features, bond_features, MAX_NB, ATOM_FDIM, BOND_FDIM
from subword_nmt.apply_bpe import BPE
import codecs
import pickle
import wget
from zipfile import ZipFile 
import os
import sys

from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import SequentialSampler
from torch import nn 

from tqdm import tqdm
import matplotlib.pyplot as plt
from time import time
from sklearn.metrics import mean_squared_error, roc_auc_score, average_precision_score, f1_score, log_loss
from lifelines.utils import concordance_index
from scipy.stats import pearsonr
import pickle 
torch.manual_seed(2)
np.random.seed(3)
import copy
from prettytable import PrettyTable

import os

from DeepPurpose.utils import *
from DeepPurpose.model_helper import Encoder_MultipleLayers, Embeddings        
from DeepPurpose.encoders import *
from DeepPurpose import DTI

import collections

In [ ]:
df = pd.read_pickle("./df.pkl")

idx_str = ['Kd', 'IC50', 'Ki','EC50','Temp']
df_want = df
convert_to_log = 0

# have at least uniprot or pubchem ID
df_want = df_want[df_want.PubChem_ID.notnull() | df_want.UniProt_ID.notnull()]
df_want = df_want[df_want.InChI.notnull()]

for label in idx_str:
    df_want[label] = df_want[label].str.replace('>', '')
    df_want[label] = df_want[label].str.replace('<', '')

y = df_want[idx_str]
for label in idx_str:
    if convert_to_log:
            print('Default set to logspace (nM -> p) for easier regression')
            y[label] = convert_y_unit(df_want[label].values, 'nM', 'p') 
    else:
            y[label] = df_want[label].values

y['pH'] = df_want['pH']
X_drugs = df_want.SMILES.values
X_targets = df_want['Target Sequence'].values
y = y.apply(pd.to_numeric, errors='coerce')
X_org = df_want['Organism']

In [ ]:
y

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer

imputer = SimpleImputer() #TODO: better imputer
y_i = imputer.fit_transform(y)
y_i = pd.DataFrame(data=y_i,columns=idx_str+['pH'])

df_data = y_i
df_data['SMILES'] = X_drugs
df_data['Target Sequence'] = X_targets
df_data['Organism'] = X_org
df_data = df_data.join(y, rsuffix='_raw')
df_data.head()

In [52]:
df_data = df_backup

In [53]:
df_backup = df_data
df_data = df_data.sample(10000)

In [54]:
import math
def magnitude(x):
    if x > 0:
        return int(math.floor(math.log10(x)))
    else:
        return 0

df_data['magic50'] = df_data['IC50'].apply(magnitude)

In [55]:
import time

drug_func_list= [smiles2morgan,trans_drug,drug2emb_encoder,calcPubChemFingerAll,smiles2mpnnfeature, smiles2daylight]
column_name = 'SMILES'
start = time.time()

for func in drug_func_list:
    save_column_name = func.__name__
    unique = pd.Series(df_data[column_name].unique()).apply(func)
    unique_dict = dict(zip(df_data[column_name].unique(), unique))
    df_data[save_column_name] = [unique_dict[i] for i in df_data[column_name]]
    end = time.time()
    print(end - start)

rdkit not found this smiles for morgan: Cc1n[nH]n(C)c1-c1cnc2c(c1)n([C@H](C1CCOCC1)c1ccccc1)c1cc(ccc21)C(C)(C)O convert to all 0 features
rdkit not found this smiles for morgan: COc1ccc2C[C@@H]3[C@]45CC[C@](OC)([C@@H]6Oc1c2[C@]46CC[N]3(C)CC1CC1)[C@@H](COCc1cc2ccccc2o1)C5 convert to all 0 features
rdkit not found this smiles for morgan: COc1ccc(cc1OC)-c1cc(C(=O)NCc2c(C)cc(C)nc2=O)c2cnn(C(C)C)c2n1 convert to all 0 features
7.282987356185913
7.7704432010650635
8.568466424942017
92.0403687953949
Molecules not found and change to zero vectors..
Molecules not found and change to zero vectors..
Molecules not found and change to zero vectors..
147.95095109939575
rdkit not found this smiles: Cc1n[nH]n(C)c1-c1cnc2c(c1)n([C@H](C1CCOCC1)c1ccccc1)c1cc(ccc21)C(C)(C)O convert to all 0 features
rdkit not found this smiles: COc1ccc2C[C@@H]3[C@]45CC[C@](OC)([C@@H]6Oc1c2[C@]46CC[N]3(C)CC1CC1)[C@@H](COCc1cc2ccccc2o1)C5 convert to all 0 features
rdkit not found this smiles: [I-] convert to all 0 features
r

In [56]:
prot_func_list = [CalculateConjointTriad, trans_protein, protein2emb_encoder,target2quasi]
column_name = 'Target Sequence'
start = time.time()

for func in prot_func_list:
    save_column_name = func.__name__
    AA = pd.Series(df_data[column_name].unique()).apply(func)
    AA_dict = dict(zip(df_data[column_name].unique(), AA))
    df_data[save_column_name] = [AA_dict[i] for i in df_data[column_name]]
    end = time.time()
    print(end - start)

2.031992197036743
2.7403295040130615
3.2397186756134033
Quasi-seq fingerprint not working for smiles: pqitlwkrplvtikiggqlkealldtgaddtvieemslpgrwkpkmiggiggfikvrqydqiiieiaghkaigtvlvgptpvniigrnlltqigatlnf convert to 0 vectors
Quasi-seq fingerprint not working for smiles: MKAFLGALEFQENEYEELKELYESLKTKQKPHTLFISCVDSRVVPNLITGTKPGELYVIRNMGNVIPPKTSHKESLSTMASIEYAIVHVGVQNLIICGHSDCGACGSTHLINDGXTKAKTPYIADWIQFLEPIKEELKNHPQFSNHFAKRSWLTERLNVRLQLNNLLSYDFIQERVVNNELKIFGWHYIIETGRIYNYNFESHFFEPIXETXKQRKSHENF convert to 0 vectors
Quasi-seq fingerprint not working for smiles: MSKSKVDNQFYSVEVGDSTFTVLKRYQNLKPIGSGAQGIVCAAYDAVLDRNVAIKKLSRPFQNQTHAKRAYRELVLMKCVNHKNIISLLNVFTPQKTLEEFQDVYLVMELMDANLCQVIQMELDHERMSYLLYQMLCGIKHLHSAGIIHRDLKPSNIVVKSDCTLKILDFGLARTAGTSFMMTPYVVTRYYRAPEVILGMGYKENVDIWSVGCIMGEMVRHKILFPGRDYIDQWNKVIEQLGTPCPEFMKKLQPTVRNYVENRPKYAGLTFPKLFPDSLFPADSEHNKLKASQARDLLSKMLVIDPAKRISVDDALQHPYINVWYDPAXXXXXDEREHTIEEWKELIYKEVMNSE convert to 0 vectors
Quasi-seq fingerprint not working for smiles: pqitlwkrplvtikiggql

In [ ]:
import sys
np.set_printoptions(threshold = sys.maxsize)

df_data.to_pickle("./df_data_small3.pkl")
df_data.to_csv('df_data_small3.csv.gz'
         , sep='|'
         , header=True
         , index=False
         , chunksize=1000
         , compression='gzip'
         , encoding='utf-8')

In [45]:
df_data_out = pd.read_csv("./df_data_small3.csv.gz"
         , sep='|'
         , compression='gzip'
         , encoding='utf-8')
                         #dtype =  {'smiles2morgan': np.ndarray, 'drug2emb_encoder': np.ndarray, 'CalculateConjointTriad': np.ndarray, 'protein2emb_encoder': np.ndarray} )

In [46]:
#turn categorical variables into numerical dummy variables for modeling

cat_list = pd.get_dummies(df_data['Organism'], prefix='var')
df_data1=df_data.join(cat_list)

In [47]:
discard=['SMILES','Target Sequence','Organism','trans_drug','trans_protein','pH_raw','Kd_raw','IC50_raw','Ki_raw','EC50_raw','Temp_raw']
df_vars=df_data1.columns.values.tolist()
to_keep=[i for i in df_vars if i not in discard]
df_final=df_data1[to_keep]
df_final.head()

,Kd,IC50,Ki,EC50,Temp,pH,magic50,smiles2morgan,drug2emb_encoder,calcPubChemFingerAll,...,var_Trypanosoma brucei brucei,var_Trypanosoma brucei rhodesiense,var_Trypanosoma cruzi,var_Vibrio fischeri,var_Vibrio harveyi,var_Woolly monkey sarcoma virus,var_Xenopus sp.,var_Yersinia enterocolitica,var_Zea mays,var_sequence
0,872669.450093,2600.0,1.941028e+06,362878.629538,27.936324,7.242816,3,[0. 0. 0. ... 0. 0. 0.],"(array([1529, 838, 1701, 973, 517, 482, 5...",[0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0...,...,0,0,0,0,0,0,0,0,0,0
1,872669.450093,43.0,1.941028e+06,362878.629538,27.936324,7.242816,1,[0. 0. 0. ... 0. 0. 0.],"(array([ 450, 205, 221, 538, 2068, 290, ...",[0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0...,...,0,0,0,0,0,0,0,0,0,0
2,872669.450093,9700.0,1.941028e+06,362878.629538,27.936324,7.242816,3,[0. 0. 0. ... 0. 0. 0.],"(array([ 54, 32, 200, 35, 295, 370, 246, 646...",[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0...,...,0,0,0,0,0,0,0,0,0,0
3,872669.450093,6.3,1.941028e+06,362878.629538,27.936324,7.400000,0,[0. 0. 0. ... 0. 1. 0.],"(array([ 43, 517, 164, 106, 168, 381, 2...",[0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 0...,...,0,0,0,0,0,0,0,0,0,0
4,872669.450093,100.0,1.941028e+06,362878.629538,27.936324,7.242816,2,[1. 0. 0. ... 0. 0. 0.],"(array([1043, 343, 158, 78, 97, 133, ...",[0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 0 0 0 0...,...,0,0,0,0,0,0,0,0,0,0


In [51]:
#X["smiles2morgan"].iloc[1]
df_backup.head()

,Kd,IC50,Ki,EC50,Temp,pH,SMILES,Target Sequence,Organism,Kd_raw,IC50_raw,Ki_raw,EC50_raw,Temp_raw,pH_raw
0,872669.450093,1.264663e+08,0.24,362878.629538,37.0,5.5,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,Human immunodeficiency virus 1,NaN,NaN,0.24,NaN,37.0,5.5
1,872669.450093,1.264663e+08,0.25,362878.629538,37.0,5.5,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(C\C=C\c2cn...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,Human immunodeficiency virus 1,NaN,NaN,0.25,NaN,37.0,5.5
2,872669.450093,1.264663e+08,0.41,362878.629538,37.0,5.5,O[C@@H]1[C@@H](O)[C@@H](Cc2ccccc2)N(CC2CC2)C(=...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,Human immunodeficiency virus 1,NaN,NaN,0.41,NaN,37.0,5.5
3,872669.450093,1.264663e+08,0.80,362878.629538,37.0,5.5,OCCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,Human immunodeficiency virus 1,NaN,NaN,0.80,NaN,37.0,5.5
4,872669.450093,1.264663e+08,0.99,362878.629538,37.0,5.5,OCCCCCN1[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)[C@@H...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,Human immunodeficiency virus 1,NaN,NaN,0.99,NaN,37.0,5.5


In [5]:
def flattener(x):
    if isinstance(x, collections.Iterable):
        return [a for i in x for a in flattener(i)]
    else:
        return [x] 

In [19]:
X = df_final.drop(["IC50","magic50"], axis=1)

In [17]:
X.head()
#dtype =  {'smiles2morgan': np.ndarray, 'drug2emb_encoder': np.ndarray, 'CalculateConjointTriad': np.ndarray, 'protein2emb_encoder': np.ndarray} 


,Kd,Ki,EC50,Temp,smiles2morgan,smiles2daylight,drug2emb_encoder,CalculateConjointTriad,protein2emb_encoder,calcPubChemFingerAll,...,var_West Nile virus,var_Woolly monkey sarcoma virus,var_Xenopus laevis,var_Xenopus sp.,var_Yersinia enterocolitica,var_Yersinia pestis,var_Zea mays,var_Zymomonas mobilis,var_[Bacteroides] pectinophilus ATCC 43243,var_sequence
1264617,872669.450093,1.941028e+06,362878.629538,27.936324,[0. 0. 0. ... 0. 0. 0.],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(array([1529, 838, 1701, 973, 517, 482, 5...",[ 4 13 9 2 7 6 1 11 11 7 6 7 15 2 11 ...,"(array([1560, 1452, 100, 3437, 45, 252, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
1533686,872669.450093,1.941028e+06,362878.629538,27.936324,[0. 0. 0. ... 0. 0. 0.],"[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...","(array([ 450, 205, 221, 538, 2068, 290, ...",[10 15 11 6 6 5 1 14 14 9 3 2 8 3 10 ...,"(array([1426, 25, 176, 176, 25, 224, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0
1125111,872669.450093,1.941028e+06,362878.629538,27.936324,[0. 0. 0. ... 0. 0. 0.],"[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","(array([ 54, 32, 200, 35, 295, 370, 246, 646...",[4 5 3 3 2 2 0 1 3 5 3 4 1 0 3 2 1 2 1 1 0 3 2...,"(array([ 186, 44, 133, 109, 1152, 143, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
225474,872669.450093,1.941028e+06,362878.629538,27.936324,[0. 0. 0. ... 0. 1. 0.],"[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...","(array([ 43, 517, 164, 106, 168, 381, 2...",[4 5 3 1 1 1 1 1 7 1 2 3 5 0 2 4 3 0 1 0 0 0 3...,"(array([ 874, 240, 125, 284, 43, 67, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0
652860,872669.450093,1.941028e+06,362878.629538,27.936324,[1. 0. 0. ... 0. 0. 0.],"[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","(array([1043, 343, 158, 78, 97, 133, ...",[ 5 10 2 0 4 0 2 4 4 6 0 4 2 0 4 ...,"(array([ 550, 487, 189, 920, 1121, 98, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0


In [24]:
typelist = []
for i in list(X):
    print(i, ':', type(X[i].iloc[1]), ':', X[i].iloc[1])
    if isinstance(X[i].iloc[1],np.ndarray):
        print(len(X[i].iloc[1]))
        typelist.extend([i]*len(X[i].iloc[1]))
    elif isinstance(X[i].iloc[1],tuple):
        for n in X[i].iloc[1]:
            if isinstance(n,np.ndarray):
                print(len(n))
                typelist.extend([i]*len(n))
    elif i[:4] == 'var_':
        typelist.append('organism')
    else:
        typelist.append(i)

Kd : <class 'numpy.float64'> : 872669.450092565
Ki : <class 'numpy.float64'> : 1941028.067873606
EC50 : <class 'numpy.float64'> : 362878.62953817454
Temp : <class 'numpy.float64'> : 27.936324372937893
smiles2morgan : <class 'list'> : ['0.', '0.', '0.', '...', '0.', '0.', '0']
smiles2daylight : <class 'numpy.ndarray'> : [1. 0. 0. ... 1. 1. 1.]
2048
drug2emb_encoder : <class 'str'> : (array([ 450,  205,  221,  538, 2068,  290,   37,   35,  168,   59,  262,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0]))
CalculateConjointTriad : <class 'list'> : ['10', '15', '11', '6', '6', '5', '1', '14', '14', '9', '3', '

var_Aspergillus oryzae : <class 'numpy.uint8'> : 0
var_Asterina pectinifera : <class 'numpy.uint8'> : 0
var_Avian erythroblastosis virus (strain ES4) : <class 'numpy.uint8'> : 0
var_Avian myeloblastosis virus : <class 'numpy.uint8'> : 0
var_Avian sarcoma virus : <class 'numpy.uint8'> : 0
var_Bacillus amyloliquefaciens : <class 'numpy.uint8'> : 0
var_Bacillus anthracis : <class 'numpy.uint8'> : 0
var_Bacillus cereus : <class 'numpy.uint8'> : 0
var_Bacillus cereus (strain ATCC 14579 / DSM 31) : <class 'numpy.uint8'> : 0
var_Bacillus clausii : <class 'numpy.uint8'> : 0
var_Bacillus lentus : <class 'numpy.uint8'> : 0
var_Bacillus licheniformis : <class 'numpy.uint8'> : 0
var_Bacillus pasteurii : <class 'numpy.uint8'> : 0
var_Bacillus sporothermodurans : <class 'numpy.uint8'> : 0
var_Bacillus subtilis : <class 'numpy.uint8'> : 0
var_Bacillus thermoproteolyticus : <class 'numpy.uint8'> : 0
var_Bacteroides thetaiotaomicron : <class 'numpy.uint8'> : 0
var_Bison bison : <class 'numpy.uint8'> : 

var_Ovis aries : <class 'numpy.uint8'> : 0
var_Paramecium tetraurelia : <class 'numpy.uint8'> : 0
var_Penicillium janthinellum : <class 'numpy.uint8'> : 0
var_Phaseolus vulgaris : <class 'numpy.uint8'> : 0
var_Photinus pyralis : <class 'numpy.uint8'> : 0
var_Photuris pennsylvanica : <class 'numpy.uint8'> : 0
var_Pichia angusta : <class 'numpy.uint8'> : 0
var_Pisum sativum : <class 'numpy.uint8'> : 0
var_Plasmodium berghei : <class 'numpy.uint8'> : 0
var_Plasmodium falciparum : <class 'numpy.uint8'> : 0
var_Plasmodium falciparum (isolate 3D7) : <class 'numpy.uint8'> : 0
var_Plasmodium falciparum (isolate FcB1 / Columbia) : <class 'numpy.uint8'> : 0
var_Plasmodium falciparum (isolate K1 / Thailand) : <class 'numpy.uint8'> : 0
var_Plasmodium vivax : <class 'numpy.uint8'> : 0
var_Pneumocystis carinii : <class 'numpy.uint8'> : 0
var_Poliovirus type 1 (strain Mahoney) : <class 'numpy.uint8'> : 0
var_Porcellium fiumanum : <class 'numpy.uint8'> : 0
var_Providencia stuartii : <class 'numpy.uint

In [20]:

from ast import literal_eval

def literal_test(x):
    try:
        return ast.literal_eval(x)
    except:
        return np.nan

#X['CalculateConjointTriad'] = X['CalculateConjointTriad'].apply(literal_test)
X.head()

,Kd,Ki,EC50,Temp,smiles2morgan,smiles2daylight,drug2emb_encoder,CalculateConjointTriad,protein2emb_encoder,calcPubChemFingerAll,...,var_West Nile virus,var_Woolly monkey sarcoma virus,var_Xenopus laevis,var_Xenopus sp.,var_Yersinia enterocolitica,var_Yersinia pestis,var_Zea mays,var_Zymomonas mobilis,var_[Bacteroides] pectinophilus ATCC 43243,var_sequence
1264617,872669.450093,1.941028e+06,362878.629538,27.936324,[0. 0. 0. ... 0. 0. 0.],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(array([1529, 838, 1701, 973, 517, 482, 5...",[ 4 13 9 2 7 6 1 11 11 7 6 7 15 2 11 ...,"(array([1560, 1452, 100, 3437, 45, 252, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
1533686,872669.450093,1.941028e+06,362878.629538,27.936324,[0. 0. 0. ... 0. 0. 0.],"[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...","(array([ 450, 205, 221, 538, 2068, 290, ...",[10 15 11 6 6 5 1 14 14 9 3 2 8 3 10 ...,"(array([1426, 25, 176, 176, 25, 224, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0
1125111,872669.450093,1.941028e+06,362878.629538,27.936324,[0. 0. 0. ... 0. 0. 0.],"[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","(array([ 54, 32, 200, 35, 295, 370, 246, 646...",[4 5 3 3 2 2 0 1 3 5 3 4 1 0 3 2 1 2 1 1 0 3 2...,"(array([ 186, 44, 133, 109, 1152, 143, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
225474,872669.450093,1.941028e+06,362878.629538,27.936324,[0. 0. 0. ... 0. 1. 0.],"[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...","(array([ 43, 517, 164, 106, 168, 381, 2...",[4 5 3 1 1 1 1 1 7 1 2 3 5 0 2 4 3 0 1 0 0 0 3...,"(array([ 874, 240, 125, 284, 43, 67, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0
652860,872669.450093,1.941028e+06,362878.629538,27.936324,[1. 0. 0. ... 0. 0. 0.],"[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","(array([1043, 343, 158, 78, 97, 133, ...",[ 5 10 2 0 4 0 2 4 4 6 0 4 2 0 4 ...,"(array([ 550, 487, 189, 920, 1121, 98, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0


In [23]:
#dtype =  {'smiles2morgan': np.ndarray, 'drug2emb_encoder': np.ndarray, 'CalculateConjointTriad': np.ndarray, 'protein2emb_encoder': np.ndarray} 
X = df_final.drop(["IC50","magic50"], axis=1)
X['CalculateConjointTriad'] = X['CalculateConjointTriad'].str.strip('[]').str.split()
X['smiles2morgan'] = X['smiles2morgan'].str.strip('[.]').str.split()
X.head()

,Kd,Ki,EC50,Temp,smiles2morgan,smiles2daylight,drug2emb_encoder,CalculateConjointTriad,protein2emb_encoder,calcPubChemFingerAll,...,var_West Nile virus,var_Woolly monkey sarcoma virus,var_Xenopus laevis,var_Xenopus sp.,var_Yersinia enterocolitica,var_Yersinia pestis,var_Zea mays,var_Zymomonas mobilis,var_[Bacteroides] pectinophilus ATCC 43243,var_sequence
1264617,872669.450093,1.941028e+06,362878.629538,27.936324,"[0., 0., 0., ..., 0., 0., 0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(array([1529, 838, 1701, 973, 517, 482, 5...","[4, 13, 9, 2, 7, 6, 1, 11, 11, 7, 6, 7, 15, 2,...","(array([1560, 1452, 100, 3437, 45, 252, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
1533686,872669.450093,1.941028e+06,362878.629538,27.936324,"[0., 0., 0., ..., 0., 0., 0]","[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...","(array([ 450, 205, 221, 538, 2068, 290, ...","[10, 15, 11, 6, 6, 5, 1, 14, 14, 9, 3, 2, 8, 3...","(array([1426, 25, 176, 176, 25, 224, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0
1125111,872669.450093,1.941028e+06,362878.629538,27.936324,"[0., 0., 0., ..., 0., 0., 0]","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","(array([ 54, 32, 200, 35, 295, 370, 246, 646...","[4, 5, 3, 3, 2, 2, 0, 1, 3, 5, 3, 4, 1, 0, 3, ...","(array([ 186, 44, 133, 109, 1152, 143, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
225474,872669.450093,1.941028e+06,362878.629538,27.936324,"[0., 0., 0., ..., 0., 1., 0]","[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...","(array([ 43, 517, 164, 106, 168, 381, 2...","[4, 5, 3, 1, 1, 1, 1, 1, 7, 1, 2, 3, 5, 0, 2, ...","(array([ 874, 240, 125, 284, 43, 67, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0
652860,872669.450093,1.941028e+06,362878.629538,27.936324,"[1., 0., 0., ..., 0., 0., 0]","[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...","(array([1043, 343, 158, 78, 97, 133, ...","[5, 10, 2, 0, 4, 0, 2, 4, 4, 6, 0, 4, 2, 0, 4,...","(array([ 550, 487, 189, 920, 1121, 98, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",...,0,0,0,0,0,0,0,0,0,0


In [ ]:
vec = X.apply(flattener,axis=1)
foo = vec.to_numpy(dtype = object)
foo2 = np.vstack(foo)

In [ ]:
from sklearn.model_selection import train_test_split
y = df_final["IC50"]
X_train, X_test, y_train, y_test = train_test_split(foo2, y, test_size=0.2, random_state=123)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_scaled = scaler.fit_transform(X_train)
test_scaled = scaler.transform(X_test)

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
tree_model = DecisionTreeRegressor()
rf_model = RandomForestRegressor()

In [ ]:
tree_model.fit(train_scaled, y_train)
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import average_precision_score
tree_mse = mean_squared_error(y_train, tree_model.predict(train_scaled))
tree_mae = mean_absolute_error(y_train, tree_model.predict(train_scaled))
from math import sqrt
print("Decision Tree training mse = ",tree_mse," & mae = ",tree_mae," & rmse = ", sqrt(tree_mse))
tree_test_mse = mean_squared_error(y_test, tree_model.predict(test_scaled))
tree_test_mae = mean_absolute_error(y_test, tree_model.predict(test_scaled))
print("Decision Tree test mse = ",tree_test_mse," & mae = ",tree_test_mae," & rmse = ", sqrt(tree_test_mse))
a = pearsonr(y_train, tree_model.predict(train_scaled))
b = pearsonr(y_test, tree_model.predict(test_scaled))
print("Decision Tree train r = ",a)
print("Decision Tree test r = ",b)

In [ ]:
importances = pd.DataFrame({'importance':np.round(tree_model.feature_importances_,3)})
out = importances.sort_values('importance',ascending=False)
importances["Type"] = typelist
type_importance = importances.groupby(by=['Type']).sum()
type_importance.sort_values('importance',ascending=False)